*Copyright (c) Microsoft Corporation. All rights reserved.*  

*Licensed under the MIT License.*

# Natural Language Inference on MultiNLI Dataset using Transformers

# Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Deep Learning Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip:** If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

The table below provides some reference running time on different machine configurations.  

|QUICK_RUN|Machine Configurations|Running time|
|:---------|:----------------------|:------------|
|True|4 **CPU**s, 14GB memory| ~ 15 minutes|
|True|1 NVIDIA Tesla K80 GPUs, 12GB GPU memory| ~ 5 minutes|
|False|1 NVIDIA Tesla K80 GPUs, 12GB GPU memory| ~ 10.5 hours|
|False|4 NVIDIA Tesla V100 GPUs, 64GB GPU memory| ~ 2.5 hours|

If you run into CUDA out-of-memory error, try reducing the `BATCH_SIZE` and `MAX_SEQ_LENGTH`, but note that model performance will be compromised. 

In [1]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True

## Summary
In this notebook, we demostrate using [BERT](https://arxiv.org/abs/1810.04805) to perform Natural Language Inference (NLI). We use the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset and the task is to classify sentence pairs into three classes: contradiction, entailment, and neutral.   
The figure below shows how [BERT](https://arxiv.org/abs/1810.04805) classifies sentence pairs. It concatenates the tokens in each sentence pairs and separates the sentences by the [SEP] token. A [CLS] token is prepended to the token list and used as the aggregate sequence representation for the classification task.
<img src="https://nlpbp.blob.core.windows.net/images/bert_two_sentence.PNG">

In [2]:
import sys, os
nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
    
from tempfile import TemporaryDirectory

import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

import torch

from utils_nlp.models.transformers.sequence_classification import Processor, SequenceClassifier
from utils_nlp.dataset.multinli import load_pandas_df
from utils_nlp.common.timer import Timer

I1107 22:32:34.756409 140217568773888 file_utils.py:39] PyTorch version 1.2.0 available.
I1107 22:32:34.801189 140217568773888 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I1107 22:32:35.187866 140217568773888 utils.py:141] NumExpr defaulting to 6 threads.


## Configurations

In [3]:
MODEL_NAME = "bert-base-uncased"

TRAIN_DATA_USED_FRACTION = 1
DEV_DATA_USED_FRACTION = 1
NUM_EPOCHS = 2
WARMUP_STEPS= 2500

if QUICK_RUN:
    TRAIN_DATA_USED_FRACTION = 0.001
    DEV_DATA_USED_FRACTION = 0.01
    NUM_EPOCHS = 1
    WARMUP_STEPS= 10

if torch.cuda.is_available():
    BATCH_SIZE = 32
else:
    BATCH_SIZE = 16

RANDOM_SEED = 42

# model configurations
TO_LOWER = True
MAX_SEQ_LENGTH = 128

# optimizer configurations
LEARNING_RATE= 5e-5

# data configurations
TEXT_COL = "text"
LABEL_COL = "gold_label"

# CACHE_DIR = TemporaryDirectory().name
CACHE_DIR = "./temp"

## Load Data
The MultiNLI dataset comes with three subsets: train, dev_matched, dev_mismatched. The dev_matched dataset are from the same genres as the train dataset, while the dev_mismatched dataset are from genres not seen in the training dataset.   
The `load_pandas_df` function downloads and extracts the zip files if they don't already exist in `local_cache_path` and returns the data subset specified by `file_split`.

In [4]:
train_df = load_pandas_df(local_cache_path=CACHE_DIR, file_split="train")
dev_df_matched = load_pandas_df(local_cache_path=CACHE_DIR, file_split="dev_matched")
dev_df_mismatched = load_pandas_df(local_cache_path=CACHE_DIR, file_split="dev_mismatched")

In [5]:
dev_df_matched = dev_df_matched.loc[dev_df_matched['gold_label'] != '-']
dev_df_mismatched = dev_df_mismatched.loc[dev_df_mismatched['gold_label'] != '-']

In [6]:
print("Training dataset size: {}".format(train_df.shape[0]))
print("Development (matched) dataset size: {}".format(dev_df_matched.shape[0]))
print("Development (mismatched) dataset size: {}".format(dev_df_mismatched.shape[0]))
print()
print(train_df[['gold_label', 'sentence1', 'sentence2']].head())

Training dataset size: 392702
Development (matched) dataset size: 9815
Development (mismatched) dataset size: 9832

   gold_label                                          sentence1  \
0     neutral  Conceptually cream skimming has two basic dime...   
1  entailment  you know during the season and i guess at at y...   
2  entailment  One of our number will carry out your instruct...   
3  entailment  How do you know? All this is their information...   
4     neutral  yeah i tell you what though if you go price so...   

                                           sentence2  
0  Product and geography are what make cream skim...  
1  You lose the things to the following level if ...  
2  A member of my team will execute your orders w...  
3                  This information belongs to them.  
4           The tennis shoes have a range of prices.  


Concatenate the first and second sentences to form the input text.

In [7]:
train_df[TEXT_COL] = list(zip(train_df['sentence1'], train_df['sentence2']))
dev_df_matched[TEXT_COL] = list(zip(dev_df_matched['sentence1'], dev_df_matched['sentence2']))
dev_df_mismatched[TEXT_COL] = list(zip(dev_df_mismatched['sentence1'], dev_df_mismatched['sentence2']))
train_df[[TEXT_COL, LABEL_COL]].head()

,text,gold_label
0,(Conceptually cream skimming has two basic dim...,neutral
1,(you know during the season and i guess at at ...,entailment
2,(One of our number will carry out your instruc...,entailment
3,(How do you know? All this is their informatio...,entailment
4,(yeah i tell you what though if you go price s...,neutral


In [8]:
train_df = train_df.sample(frac=TRAIN_DATA_USED_PERCENT).reset_index(drop=True)
dev_df_matched = dev_df_matched.sample(frac=DEV_DATA_USED_PERCENT).reset_index(drop=True)
dev_df_mismatched = dev_df_mismatched.sample(frac=DEV_DATA_USED_PERCENT).reset_index(drop=True)

NameError: name 'TRAIN_DATA_USED_PERCENT' is not defined

In [ ]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df[LABEL_COL])
num_labels = len(np.unique(train_labels))

## Tokenize and Preprocess
Before training, we tokenize the sentence texts and convert them to lists of tokens. The following steps instantiate a BERT tokenizer given the language, and tokenize the text of the training and testing sets.

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
processor = Processor(model_name=MODEL_NAME, cache_dir=CACHE_DIR, to_lower=TO_LOWER)
train_dataset = processor.preprocess_sentence_pair(
    train_df[TEXT_COL], train_labels, max_len=MAX_SEQ_LENGTH
)
dev_dataset_matched = processor.preprocess_sentence_pair(dev_df_matched[TEXT_COL], None, max_len=MAX_SEQ_LENGTH)
dev_dataset_mismatched = processor.preprocess_sentence_pair(dev_df_mismatched[TEXT_COL], None, max_len=MAX_SEQ_LENGTH)

In addition, we perform the following preprocessing steps in the cell below:

* Convert the tokens into token indices corresponding to the BERT tokenizer's vocabulary
* Add the special tokens [CLS] and [SEP] to mark the beginning and end of a sentence
* Pad or truncate the token lists to the specified max length
* Return mask lists that indicate paddings' positions
* Return token type id lists that indicate which sentence the tokens belong to

*See the original [implementation](https://github.com/google-research/bert/blob/master/run_classifier.py) for more information on BERT's input format.*

## Train and Predict

### Create Classifier

In [ ]:
classifier = SequenceClassifier(
    model_name=MODEL_NAME, num_labels=num_labels, cache_dir=CACHE_DIR
)

### Train Classifier

In [ ]:
with Timer() as t:
    classifier.fit(
            train_dataset,
            num_epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            learning_rate=LEARNING_RATE,
            warmup_steps=WARMUP_STEPS,
        )

print("Training time : {:.3f} hrs".format(t.interval / 3600))

### Predict on Test Data

In [ ]:
with Timer() as t:
    predictions_matched = classifier.predict(dev_dataset_matched, batch_size=BATCH_SIZE)
print("Prediction time : {:.3f} hrs".format(t.interval / 3600))

In [ ]:
with Timer() as t:
    predictions_mismatched = classifier.predict(dev_dataset_mismatched, batch_size=BATCH_SIZE)
print("Prediction time : {:.3f} hrs".format(t.interval / 3600))

## Evaluate

In [ ]:
predictions_matched = label_encoder.inverse_transform(predictions_matched)
print(classification_report(dev_df_matched[LABEL_COL], predictions_matched, digits=3))

In [ ]:
predictions_mismatched = label_encoder.inverse_transform(predictions_mismatched)
print(classification_report(dev_df_mismatched[LABEL_COL], predictions_mismatched, digits=3))